<a href="https://colab.research.google.com/github/ccarpenterg/LearningMXNet/blob/master/03_introduction_to_convnets_with_mxnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Convolutional Neural Network with MXNet

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [2]:
!pip install mxnet-cu100

     |████████████████████████████████| 540.1MB 40kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
from __future__ import print_function

import mxnet as mx
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn


print(mx.__version__)

1.5.0


### MNIST Dataset

In [4]:
def data_convention_normalization(data):
    return nd.moveaxis(data, 2, 0).astype('float32') / 255

train_data = gluon.data.vision.MNIST(train=True).transform_first(data_convention_normalization)
valid_data = gluon.data.vision.MNIST(train=False).transform_first(data_convention_normalization)

In [0]:
train_loader = gluon.data.DataLoader(train_data, shuffle=True, batch_size=64)
valid_loader = gluon.data.DataLoader(valid_data, shuffle=False, batch_size=64)

In [7]:
net = nn.Sequential()

net.add(nn.Conv2D(channels=32, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2),
        nn.Conv2D(channels=64, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2),
        nn.Conv2D(channels=64, kernel_size=3, activation='relu'),
        nn.Dense(64, activation='relu'),
        nn.Dense(10))

net

Sequential(
  (0): Conv2D(None -> 32, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (2): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (4): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (5): Dense(None -> 64, Activation(relu))
  (6): Dense(None -> 10, linear)
)

In [9]:
ctx = mx.gpu(0) if mx.context.num_gpus() > 0 else mx.cpu(0)
net.initialize(mx.init.Xavier(), ctx=ctx)
net.summary(nd.zeros((1, 1, 28, 28), ctx=ctx))

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                              (1, 1, 28, 28)               0
        Activation-1                     <Symbol conv0_relu_fwd>               0
        Activation-2                             (1, 32, 26, 26)               0
            Conv2D-3                             (1, 32, 26, 26)             320
         MaxPool2D-4                             (1, 32, 13, 13)               0
        Activation-5                     <Symbol conv1_relu_fwd>               0
        Activation-6                             (1, 64, 11, 11)               0
            Conv2D-7                             (1, 64, 11, 11)           18496
         MaxPool2D-8                               (1, 64, 5, 5)               0
        Activation-9                     <Symbol conv2_relu_fwd>               0
       Activation-10        

In [0]:
trainer = gluon.Trainer(
    params=net.collect_params(),
    optimizer='sgd',
    optimizer_params={'learning_rate': 0.04},
)